<a href="https://colab.research.google.com/github/KSY1526/myblog/blob/master/_notebooks/2021-12-28-Do_natural_language6.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# "[Do it 자연어] 6. 질문에 답하는 웹 서비스 만들기"
- author: Seong Yeon Kim 
- categories: [book, jupyter, Do it, natural language, BERT, tokenizer, web, answer]

# 모델 환경설정

In [1]:
!pip install ratsnlp

     |████████████████████████████████| 53 kB 1.4 MB/s 
     |████████████████████████████████| 57 kB 4.8 MB/s 
     |████████████████████████████████| 806 kB 16.6 MB/s 
     |████████████████████████████████| 2.8 MB 44.7 MB/s 
     |████████████████████████████████| 829 kB 44.4 MB/s 
     |████████████████████████████████| 132 kB 45.4 MB/s 
     |████████████████████████████████| 332 kB 53.9 MB/s 
     |████████████████████████████████| 636 kB 53.6 MB/s 
     |████████████████████████████████| 3.3 MB 25.7 MB/s 
     |████████████████████████████████| 61 kB 550 kB/s 
     |████████████████████████████████| 895 kB 51.6 MB/s 
     |████████████████████████████████| 1.1 MB 44.2 MB/s 
     |████████████████████████████████| 96 kB 6.2 MB/s 
     |████████████████████████████████| 160 kB 51.0 MB/s 
     |████████████████████████████████| 271 kB 50.6 MB/s 
     |████████████████████████████████| 192 kB 51.1 MB/s 
  Created wheel for future: filename=future-0.18.2-py3-none-any.whl size=491070 

In [2]:
from google.colab import drive
drive.mount('/gdrive', force_remount= True)

Mounted at /gdrive


In [6]:
from torch.cuda import is_available
import torch
from ratsnlp.nlpbook.qa import QATrainArguments

args = QATrainArguments(
    pretrained_model_name= 'beomi/kcbert-base', # 프리트레인 모델
    downstream_corpus_name='korquad-v1', # 입력 데이터
    downstream_model_dir= '/gdrive/My Drive/nlpbook/checkpoing-qa', # 중간저장위치
    max_seq_length=128, # 입력문장 최대 길이(질문+지문)
    max_query_length=32, # 질문 최대 길이
    doc_stride = 64, # 지문에서 몇개 토큰을 슬라이딩 해갈지? 결정
    batch_size = 32 if torch.cuda.is_available() else 4,
    learning_rate=5e-5,
    epochs = 3,
    tpu_cores=0 if torch.cuda.is_available() else 8,
    seed = 7,
)

kcbert-base 모델로 프리트레인을 하였습니다.

LG CNS가 공개한 korqued 1.0 데이터를 활용해 모델을 구축하겠습니다.

여기서 doc_stride은 만약 입력문장이 128보다 클 경우 지문 앞 부분을 입력값(64)만큼 제거하고 잘린 뒷부분을 가져와 붙입니다.

In [7]:
from ratsnlp import nlpbook
nlpbook.set_seed(args)
nlpbook.set_logger(args)

INFO:ratsnlp:Training/evaluation parameters QATrainArguments(pretrained_model_name='beomi/kcbert-base', downstream_corpus_name='korquad-v1', downstream_corpus_root_dir='/root/Korpora', downstream_model_dir='/gdrive/My Drive/nlpbook/checkpoing-qa', max_seq_length=128, doc_stride=64, max_query_length=32, threads=4, cpu_workers=2, save_top_k=1, monitor='min val_loss', seed=7, overwrite_cache=False, force_download=False, test_mode=False, learning_rate=5e-05, epochs=3, batch_size=32, fp16=False, tpu_cores=0, tqdm_enabled=True)
INFO:ratsnlp:Training/evaluation parameters QATrainArguments(pretrained_model_name='beomi/kcbert-base', downstream_corpus_name='korquad-v1', downstream_corpus_root_dir='/root/Korpora', downstream_model_dir='/gdrive/My Drive/nlpbook/checkpoing-qa', max_seq_length=128, doc_stride=64, max_query_length=32, threads=4, cpu_workers=2, save_top_k=1, monitor='min val_loss', seed=7, overwrite_cache=False, force_download=False, test_mode=False, learning_rate=5e-05, epochs=3, bat

set seed: 7


랜덤시드와 로거를 설정했습니다.

# 말뭉치 내려받기

In [8]:
nlpbook.download_downstream_dataset(args)

Downloading: 38.5MB [00:00, 71.7MB/s]
Downloading: 3.88MB [00:00, 60.6MB/s]                  


downstream_corpus_name='korquad-v1' 로 선언된 데이터를 내려받습니다.

In [9]:
from transformers import BertTokenizer
tokenizer = BertTokenizer.from_pretrained(
    args.pretrained_model_name,
    do_lower_case = False,
)

Downloading:   0%|          | 0.00/250k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/49.0 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/619 [00:00<?, ?B/s]

pretrained_model_name= 'beomi/kcbert-base' 로 프리트레인을 해서 토크나이저를 생성합니다.

# 학습 데이터 구축

In [12]:
from ratsnlp.nlpbook.qa import KorQuADV1Corpus, QADataset
corpus = KorQuADV1Corpus() # JSON 포멧의 KorQuAD 데이터를 읽어옴.
train_dataset = QADataset(
    args = args,
    corpus = corpus,
    tokenizer = tokenizer,
    mode = 'train'
)

INFO:ratsnlp:Creating features from train dataset file at /root/Korpora/korquad-v1
INFO:ratsnlp:Creating features from train dataset file at /root/Korpora/korquad-v1
convert squad examples to features: 100%|██████████| 57688/57688 [07:46<00:00, 123.67it/s]
INFO:ratsnlp:*** Example ***
INFO:ratsnlp:*** Example ***
INFO:ratsnlp:question & context: [CLS] 바 ##그 ##너 ##는 괴 ##테 ##의 파 ##우스 ##트를 읽고 무엇을 쓰고 ##자 했 ##는가 ? [SEP] 18 ##3 ##9년 바 ##그 ##너 ##는 괴 ##테 ##의 파 ##우스 ##트 ##을 처음 읽고 그 내용 ##에 마음이 끌려 이를 소재 ##로 해서 하나의 교 ##향 ##곡 ##을 쓰 ##려는 뜻을 갖 ##는다 . 이 시기 바 ##그 ##너 ##는 18 ##3 ##8년 ##에 빛 독 ##촉 ##으로 산 ##전 ##수 ##전을 다 걲 ##은 상황이 ##라 좌 ##절 ##과 실망 ##에 가득 ##했 ##으며 메 ##피 ##스 ##토 ##펠 ##레스 ##를 만나는 파 ##우스 ##트 ##의 심 ##경 ##에 공감 ##했다고 한다 . 또한 파리 ##에서 아 ##브 ##네 ##크 ##의 지휘 ##로 파리 음악 ##원 관 ##현 ##악 ##단이 연 ##주 ##하는 베 ##토 [SEP]
INFO:ratsnlp:question & context: [CLS] 바 ##그 ##너 ##는 괴 ##테 ##의 파 ##우스 ##트를 읽고 무엇을 쓰고 ##자 했 ##는가 ? [SEP] 18 ##3 ##9년 바 ##그 ##너 ##는 괴 ##테 ##의 파 ##우스 ##트 ##을 처음 읽고 그 내용 ##에 마음이 끌려 이를 소재 ##로 해서 하나의 교 

KorQuADV1Corpus 클레스로 KorQuAD 데이터를 QAExample 형식으로 읽어옵니다.

여기서 QAExample은 질문, 지문, 정답, 정답의 시작 인덱스로 구성됩니다.

QADataset에서는 QAExample 형식으로 입력된 값들을 QAFeatures 형태로 가공해서 모델이 학습할 수 있도록 변환해줍니다.

여기서 QAFeatures은 input_ids(입력 시퀀스), attention_mask(패딩 여부), token_type_ids(세그먼트 정보, 질문은 0/지문은 1), start(or end)_positions(정답 문장 위치) 총 5개로 구성됩니다.

In [18]:
from torch.utils.data import DataLoader, RandomSampler
train_dataloader = DataLoader(
    train_dataset,
    batch_size = args.batch_size,
    sampler = RandomSampler(train_dataset, replacement = False),
    collate_fn = nlpbook.data_collator,
    drop_last = False,
    num_workers = args.cpu_workers,
)

데이터 로더를 구축했습니다. 일정 배치 크기만큼 뽑아서 모델을 학습시킬 수 있어요.

# 평가 데이터 구축

In [20]:
from torch.utils.data import SequentialSampler

val_dataset = QADataset(
    args = args,
    corpus = corpus,
    tokenizer = tokenizer,
    mode = 'val',
)

val_dataloader = DataLoader(
    val_dataset,
    batch_size = args.batch_size,
    sampler = SequentialSampler(val_dataset),
    collate_fn = nlpbook.data_collator,
    drop_last=False,
    num_workers = args.cpu_workers,
)

INFO:ratsnlp:Loading features from cached file /root/Korpora/korquad-v1/cached_val_BertTokenizer_maxlen-128_maxquerylen-32_docstride-64_korquad-v1_question-answering [took 4.527 s]
INFO:ratsnlp:Loading features from cached file /root/Korpora/korquad-v1/cached_val_BertTokenizer_maxlen-128_maxquerylen-32_docstride-64_korquad-v1_question-answering [took 4.527 s]


테스트용 데이터와 로더를 만들었습니다.

# 모델 학습

In [22]:
from transformers import BertConfig, BertForQuestionAnswering
pretrained_model_config = BertConfig.from_pretrained(
    args.pretrained_model_name,
)

model = BertForQuestionAnswering.from_pretrained(
    args.pretrained_model_name,
    config = pretrained_model_config,
)

Downloading:   0%|          | 0.00/438M [00:00<?, ?B/s]

Some weights of the model checkpoint at beomi/kcbert-base were not used when initializing BertForQuestionAnswering: ['cls.seq_relationship.weight', 'cls.predictions.decoder.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.bias', 'cls.predictions.transform.LayerNorm.bias', 'cls.seq_relationship.bias', 'cls.predictions.decoder.weight', 'cls.predictions.transform.dense.bias']
- This IS expected if you are initializing BertForQuestionAnswering from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForQuestionAnswering from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForQuestionAnswering were not initialized from the model c

윗 코드는 BERT 모델을 프리트레인 했고, 아랫 코드는 윗 코드로 만든 모델에 질의응답용 태스크 모듈을 덧붙힌 모델을 만들었습니다.

지금 만든 모델은 빈 껍데기(초기 모델) 입니다.

In [23]:
from ratsnlp.nlpbook.qa import QATask
task = QATask(model, args)
trainer = nlpbook.get_trainer(args)

GPU available: True, used: True
TPU available: False, using: 0 TPU cores


옵티마이저와 러닝레이트를 정한 태스크를 만들어줍니다.

그 후 트레이너를 정의하는데 모델에 세부 설정을 해줍니다.(GPU, 체크포인트 등)

In [ ]:
trainer.fit(
    task,
    train_dataloader = train_dataloader,
    val_dataloaders = val_dataloader,
)

LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]

  | Name  | Type                     | Params
---------------------------------------------------
0 | model | BertForQuestionAnswering | 108 M 
---------------------------------------------------
108 M     Trainable params
0         Non-trainable params
108 M     Total params
433.318   Total estimated model params size (MB)


Training: 0it [00:00, ?it/s]

# 질문에 답하는 웹 서비스 만들기

In [ ]:
from ratsnlp.nlpbook.ner import QADeployArguments
args = QADeployArguments(
    pretrained_model_name= 'beomi/kcbert-base',
    downstream_model_dir='/gdrive/My Drive/nlpbook/checkpoint-ner',
    max_seq_length=128,
    max_query_length = 32,
)

In [ ]:
from transformers import BertTokenizer
tokenizer = BertTokenizer.from_pretrained(
    args.pretrained_model_name,
    do_lower_case = False,
)

인퍼런스 설정, 토크나이저 초기화를 합니다.

In [ ]:
import torch
from transformers import BertConfig, BertForQuestionAnswering
fine_tuned_model_ckpt = torch.load(
    args.downstream_model_checkpoint_fpath,
    map_location = torch.device('cpu'),
)

pretrained_model_config = BertConfig.from_pretrained(
    args.pretrained_model_name,
)

model = BertForQuestionAnswering(pretrained_model_config)

체크포인트 로드, BERT 설정 로드, BERT 모델 초기화를 했습니다.

In [ ]:
model.load_state_dict({k.replace('model.',''): v for k, v in fine_tuned_model_ckpt['state_dict'].ithes()})
model.eval()

체크포인트를 주입하고 모델을 평가모드로 바꿉니다.

In [ ]:
def inference_fn(question, context):
    if question and context:
        # 질문을 토큰화하고 인덱싱. max_query_length보다 길면 자르기
        truncated_query = tokenizer.encode(
            question,
            add_special_tokens = False,
            truncation = True,
            max_length = args.max_query_length,
        )

        # 앞서 처리한 질문을 지문과 함께 토큰화하고 인덱싱. max_seq_length보다 길면 자르기
        inputs = tokenizer.encode_plus(
            text = truncated_query,
            text_pair = context,
            truncation = 'only_second',
            padding = 'max_length',
            max_length = args.max_seq_length,
            return_token_type_ids = True,
        )

        with torch.no_grad():
            outputs = model(**{k: torch.tensor([v]) for k, v in inputs.items()})
            start_pred = outputs.start_logits.argmax(dim = -1).item()
            end_pred = outputs.end_logits.argmax(dim = -1).item()
            # 정답 시작부터 끝까지 토큰들을 이어붙여서 정답 만들기
            pred_text = tokenizer.decode(inputs['input_ids'][start_pred:end_pred+1])

    else:
        pred_text = ''
    return{
        'question':question,
        'context':context,
        'answer':pred_text,
    }

실제 값이 들어왔을때 처리해주는 함수입니다.

In [ ]:
from ratsnlp.nlpbook.ner import get_web_service_app
app = get_web_service_app(inference_fn)
app.run()

웹 서비스를 구현합니다.

(이쪽 개념을 잘 몰라서 실제 구현에는 실패했습니다.)